# **Sudoku Solver**

# Importing Required Libraries

In [2]:
import copy
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from my_model import get_my_model
import time

# **Method to read and collect Data from Dataset**

In [3]:
def get_data(file): 

    data = pd.read_csv(file)

    feat_raw = data['quizzes']
    label_raw = data['solutions']

    feat = []
    label = []

    for i in feat_raw:
    
        x = np.array([int(j) for j in i]).reshape((9,9,1))
        feat.append(x)
    
    feat = np.array(feat)
    feat = feat/9
    feat -= .5    
    
    for i in label_raw:
    
        x = np.array([int(j) for j in i]).reshape((81,1)) - 1
        label.append(x)   
    
    label = np.array(label)
    
    del(feat_raw)
    del(label_raw)    

    x_train, x_test, y_train, y_test = train_test_split(feat, label, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test

In [ ]:
pip install tensorflow

In [ ]:
pip install keras

In [29]:
pip install scikilearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=5ff6d07c97e68e633260d64acdca3d2abd295954e3307ed37d6f448619d2ad14
  Stored in directory: c:\users\prince\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


## Load the data

In [4]:
x_train, x_test, y_train, y_test = get_data('sudoku.csv')

## Train your own Model

In [39]:
my_model = get_my_model()

adam = keras.optimizers.Adam(lr=.001)
my_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

my_model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
25000/25000 [==============================] - 1846s 74ms/step - loss: 0.4464
Epoch 2/2
25000/25000 [==============================] - 1643s 66ms/step - loss: 0.3598


## Or load pretrained model

In [5]:
my_model = keras.models.load_model('model/sudoku.model')

## Solve Sudoku by filling blank positions one by one

In [6]:
def sudoku_result(flag):
    
    feature = copy.copy(flag)
    
    while(1):
    
        output = my_model.predict(feature.reshape((1,9,9,1)))  
        output = output.squeeze()

        predict_ = np.argmax(output, axis=1).reshape((9,9))+1 
        problem = np.around(np.max(output, axis=1).reshape((9,9)), 2) 
        
        feature = ((feature+.5)*9).reshape((9,9))
        mask = (feature==0)
     
        if(mask.sum()==0):
            break
            
        problem_new = problem*mask
    
        ind = np.argmax(problem_new)
        x, y = (ind//9), (ind%9)

        value = predict_[x][y]
        feature[x][y] = value
        feature = (feature/9)-.5
    
    return predict_

## Testing 100 games

In [7]:
def accuracy_test(features, y_out):
    
    right_ones = 0
    
    for i,feature in enumerate(features):
        
        predict = sudoku_result(feature)
        
        true = y_out[i].reshape((9,9))+1
        
        if(abs(true - predict).sum()==0):
            right_ones += 1
        
    print(right_ones/features.shape[0])

In [7]:
accuracy_test(x_test[:100], y_test[:100])

1.0


## Test your own game

In [8]:
def sudoku_solver(input_puzzle):
    
    input_puzzle = input_puzzle.replace('\n', '')
    input_puzzle = input_puzzle.replace(' ', '')
    input_puzzle = np.array([int(j) for j in input_puzzle]).reshape((9,9,1))
    input_puzzle = (input_puzzle/9)-.5
    input_puzzle = sudoku_result(input_puzzle)
    return input_puzzle

In [9]:
puzzle = '''
          1 2 3 0 8 5 4 0 0
          0 0 0 0 3 4 0 2 6
          0 0 6 0 1 0 0 0 3
          0 0 7 9 2 0 0 0 0
          3 9 0 0 0 0 0 6 2
          0 0 5 4 7 3 0 0 9
          0 7 2 0 0 0 9 0 1
          0 0 0 1 0 7 0 4 0
          9 5 0 3 4 2 0 0 8
      '''
start=time.time()
puzzle = sudoku_solver(puzzle)

print('solved puzzle:\n')
print(puzzle)
end=time.time()
print(end-start)

solved puzzle:

[[1 2 3 6 8 5 4 9 7]
 [5 8 9 7 3 4 1 2 6]
 [7 4 6 2 1 9 5 8 3]
 [8 1 7 9 2 6 3 5 4]
 [3 9 4 8 5 1 7 6 2]
 [2 6 5 4 7 3 8 1 9]
 [4 7 2 5 6 8 9 3 1]
 [6 3 8 1 9 7 2 4 5]
 [9 5 1 3 4 2 6 7 8]]
2.2502987384796143


In [10]:
np.sum(puzzle, axis=1)

array([45, 45, 45, 45, 45, 45, 45, 45, 45], dtype=int64)

In [13]:
puzzle = '''
          0 8 0 0 3 2 0 0 1
          7 0 3 0 8 0 0 0 2
          5 0 0 0 0 7 0 3 0
          0 5 0 0 0 1 9 7 0
          6 0 0 7 0 9 0 0 8
          0 4 7 2 0 0 0 5 0
          0 2 0 6 0 0 0 0 9
          8 0 0 0 9 0 3 0 5
          3 0 0 8 2 0 0 1 0
      '''
start=time.time()
puzzle = sudoku_solver(puzzle)

print('solved puzzle:\n')
print(puzzle)
end=time.time()
print(end-start)

solved puzzle:

[[4 8 9 5 3 2 7 6 1]
 [7 1 3 4 8 6 5 9 2]
 [5 6 2 9 1 7 8 3 4]
 [2 5 8 3 4 1 9 7 6]
 [6 3 1 7 5 9 2 4 8]
 [9 4 7 2 6 8 1 5 3]
 [1 2 5 6 7 3 4 8 9]
 [8 7 6 1 9 4 3 2 5]
 [3 9 4 8 2 5 6 1 7]]
1.7136952877044678
